## Classification (b)

In [1]:
from neuralnet import *
from random import shuffle
import numpy as np

# Load the configuration.
config = load_config("./data")

# Create the model
#model  = Neuralnetwork(config)
# Load the data
#x_train, y_train = load_data(path="./data", mode="train") # output normalized and one hot encoding
#x_test, y_test = load_data(path="./data", mode="test")# output normalized and one hot encoding

# Use 10 examples one from each category

In [2]:

# Data loading and spliting
X_b, y_b, X_stats = load_data(path="./data", mode="train")

X_train = []
y_train = []
class_num = list(range(10))


X_train, y_train = [], []
for idx in range(y_b.shape[0]):
    if len(class_num) == 0:
        break
    indice = onehot_decode(y_b[idx])
    
    if indice in class_num:
        X_train.append(X_b[idx])
        y_train.append(y_b[idx])
        class_num.remove(indice)
        
        
X_train = np.array(X_train)        
y_train = np.array(y_train)        

In [3]:
# Load model para
config_prob_b = {}
config_prob_b['layer_specs'] = [3072, 10, 10, 10]
config_prob_b['activation'] = 'ReLU'
config_prob_b['learning_rate'] = 0.15 
config_prob_b['batch_size'] = 128 
config_prob_b['epochs'] = 100  
config_prob_b['early_stop'] = True 
config_prob_b['early_stop_epoch'] = 5  
config_prob_b['L2_penalty'] = 0  
config_prob_b['momentum'] = True  
config_prob_b['momentum_gamma'] = 0.9  

model_b = Neuralnetwork(config_prob_b)
model_b.forward(X_train, y_train)
model_b.backward()


array([[ -9.79886062,  13.59701097,  18.47396975, ...,  22.88539403,
         -0.78263025, -13.16211926],
       [ -9.12626439,  21.62410503,  17.24807043, ...,  23.84501683,
        -13.85896364, -21.88255205],
       [  1.20463813,   6.53872047,   7.94031121, ...,  29.2017095 ,
        -19.80950099,  31.39316161],
       ...,
       [  7.02411135,  -7.06996153, -16.45616821, ..., -32.75096404,
         12.25307948, -47.16858759],
       [ -0.86306036,   4.52222992,   8.01977243, ...,  14.15783482,
        -19.00879671, -15.9398642 ],
       [ -0.9417694 ,  -1.50564644,  -0.49143815, ...,   3.30761467,
         -7.81585914,  34.73061464]])

In [4]:
# d_b d_w comparison

## Part (b) Estimation of bias weight and weight
def Num_Est_b(model, layer, eps, input_idx, output_idx):
    layer.b[0][output_idx] += eps # E(w+e)
    loss_1 = model.forward(X_train, y_train)
    layer.b[0][output_idx] -= 2*eps # E(w-e)
    loss_2 = model.forward(X_train, y_train)
    layer.b[0][output_idx] += eps # back to normal
    return (loss_1 - loss_2) / (2 * eps) # Numerical estimation for dEdW

def Num_Est_w(model, layer, eps, input_idx, output_idx):
    layer.w[input_idx][output_idx] += eps # E(w+e)
    loss_1 = model.forward(X_train, y_train)
    layer.w[input_idx][output_idx] -= 2*eps # E(w-e)
    loss_2 = model.forward(X_train, y_train)
    layer.w[input_idx][output_idx] += eps # back to normal
    return (loss_1 - loss_2) / (2 * eps) # Numerical estimation for dEdW

eps = 1e-2 # difference expectation to 10e-4
db = []
dw = []
db_est = []
dw_est = []

for layer in model_b.layers:
    if isinstance(layer, Activation):
        continue
    db_est.append(Num_Est_b(model_b, layer, eps, 0, 1))
    db.append(layer.d_b[1]) 
    dw_est.append([Num_Est_w(model_b, layer, eps, 0, 0), Num_Est_w(model_b,layer, eps, 0, 1)])
    dw.append([layer.d_w[0][0] ,layer.d_w[0][1]])
        
print('(db - db_est): {}'.format(np.array(db)-np.array(db_est)))
print('(dw - dw_est): {}'.format(np.array(dw)-np.array(dw_est)))
print('(db ): {}'.format(np.array(db)))
print('(db_est): {}'.format(np.array(db_est)))
print('(dw ): {}'.format(np.array(dw)))
print('(dw_est): {}'.format(np.array(dw_est)))

(db - db_est): [-0.66830518  0.28061107  0.10005199]
(dw - dw_est): [[ -0.42866988   1.14612641]
 [ -7.76988313  11.99803262]
 [-18.81964457   5.93045641]]
(db ): [-0.66830518  0.28061107  0.10005199]
(db_est): [0. 0. 0.]
(dw ): [[ -0.42866988   1.14612641]
 [ -7.76988313  11.99803262]
 [-18.81964457   5.93045641]]
(dw_est): [[0. 0.]
 [0. 0.]
 [0. 0.]]
